<a href="https://colab.research.google.com/github/budennovsk/Pandas/blob/master/mesa_ABM_ipywidgets_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mesa ipywidgets


  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.9 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [8]:
df = pd.read_excel('/content/mat_y_40_sku.xlsx')

In [38]:
df['Analytic1_encoded'] = df['Analytic1'].astype('category').cat.codes
df_12=df[df['Month']==12]
df_12.head(3)

,Month,Year,PRICE PER UNIT,SALES VOL,SHARE VOL,Analytic4,Analytic1,Brand,Sku_name,Analytic1_encoded
11,12,2023,674.086394,65505.60,0.739140,Слайсы,150 г,Hochland,Сыр плавленый Hochland Чизбургер Слайсы 150 г ...,2
23,12,2023,469.424564,76620.40,0.244997,Ванночки,400 г,LiebenDorf,Сыр плавленый LiebenDorf Сливочный Ванночки 40...,6
35,12,2023,518.237679,86173.56,0.635385,Блочки,90 г,Карат,"Сыр плавленый Карат Дружба Блочки 90 г 45,0%",8


In [97]:
import pandas as pd
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
import random

# Агент-потребитель
class ConsumerAgent(Agent):
    def __init__(self, unique_id, model, price_sensitivity, popularity_sensitivity, analytic_preference):
        super().__init__(unique_id, model)
        self.price_sensitivity = price_sensitivity
        self.popularity_sensitivity = popularity_sensitivity
        self.analytic_preference = analytic_preference

    def step(self):
        best_product = None
        best_score = -float('inf')

        # Проходим по каждому продукту в модели
        for idx, product in self.model.products.iterrows():
            # Расчет оценки цены
            price_score = product['PRICE PER UNIT'] * self.price_sensitivity

            # Оценка популярности (объем продаж)
            sales_score = (product['SALES VOL'] / self.model.max_sales_vol) * self.popularity_sensitivity

            # Оценка аналитического параметра
            analytic_score = abs(product['Analytic1_encoded'] - self.analytic_preference)

            # Общая оценка продукта
            total_score = price_score + sales_score + analytic_score

            if total_score > best_score:
                best_score = total_score
                best_product = idx

        # Если агент выбрал продукт, увеличиваем его долю на рынке
        if best_product is not None:
            self.model.products.at[best_product, 'SHARE VOL'] += 1

# Модель рынка
class MarketModel(Model):
    def __init__(self, N, width, height, product_data):
        super().__init__()
        self.num_agents = N  # Количество потребителей
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)

        # Продукты (SKU) представлены как DataFrame
        self.products = product_data.copy()
        self.products['SHARE VOL'] = 0  # Инициализация доли рынка
        self.max_sales_vol = self.products['SALES VOL'].max()

        # Создаем агентов-потребителей
        for i in range(self.num_agents):
            price_sensitivity = best_params[0]*random.uniform(0.5, 10.5)#   -1.0933  #random.uniform(0.5, 1.5)
            popularity_sensitivity = best_params[1]*random.uniform(0.1, 10.0) #1.4316 #random.uniform(0.1, 1.0)
            analytic_preference = best_params[2]*random.uniform(0, 10)  #3.8933 #random.uniform(0, 1)  # Предпочтение по параметру Analytic1_encoded
            consumer = ConsumerAgent(i, self, price_sensitivity, popularity_sensitivity, analytic_preference)
            self.schedule.add(consumer)

    def step(self):
        # Выполняем один шаг моделирования
        self.products['SHARE VOL'] = 0  # Сбрасываем долю рынка для каждого SKU
        self.schedule.step()

        # Меняем цены продуктов для имитации рыночных изменений
        self.products['PRICE PER UNIT'] *= self.products['PRICE PER UNIT'].apply(lambda x: random.uniform(0.95, 1.05))

        # Динамическое удаление продуктов
        if random.random() < 0.1:  # С вероятностью 30% удаляем продукт
            if not self.products.empty:
                del_sku = random.randint(0, len(self.products) - 1)
                print('Удалили', self.products.iloc[del_sku]['Sku_name'])
                self.products = self.products.drop(self.products.index[del_sku])


initial_products_df = df_12.head(5)
market = MarketModel(10, 10, 10, initial_products_df)

# Моделируем 10 временных шагов
for i in range(10):
    print(f"Step {i+1}")
    market.step()

    # Печать долей рынка каждого SKU
    print(market.products[['Sku_name', 'SHARE VOL']])


Step 1
                                             Sku_name  SHARE VOL
11  Сыр плавленый Hochland Чизбургер Слайсы 150 г ...          0
23  Сыр плавленый LiebenDorf Сливочный Ванночки 40...          0
35       Сыр плавленый Карат Дружба Блочки 90 г 45,0%         10
47  Сыр плавленый Viola Сливочный Ванночки 400 г 5...          0
59  Сыр плавленый Свежий ряд Янтарь Ванночки 400 г...          0
Step 2
                                             Sku_name  SHARE VOL
11  Сыр плавленый Hochland Чизбургер Слайсы 150 г ...          0
23  Сыр плавленый LiebenDorf Сливочный Ванночки 40...          0
35       Сыр плавленый Карат Дружба Блочки 90 г 45,0%         10
47  Сыр плавленый Viola Сливочный Ванночки 400 г 5...          0
59  Сыр плавленый Свежий ряд Янтарь Ванночки 400 г...          0
Step 3
                                             Sku_name  SHARE VOL
11  Сыр плавленый Hochland Чизбургер Слайсы 150 г ...          0
23  Сыр плавленый LiebenDorf Сливочный Ванночки 40...          0
35  

In [86]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Пример objective_function, основанной на условиях
def objective_function(params, product_data, max_sales_vol):
    price_sensitivity, popularity_sensitivity, analytic_preference = params

    total_score = 0
    # Проходим по каждому продукту в данных
    for _, product in product_data.iterrows():
        price_score = -product['PRICE PER UNIT'] * price_sensitivity
        sales_score = (product['SALES VOL'] / max_sales_vol) * popularity_sensitivity
        analytic_score = -abs(product['Analytic1_encoded'] - analytic_preference)

        product_score = price_score + sales_score + analytic_score
        total_score += product_score

    # Возвращаем отрицательное значение для минимизации (максимизация score)
    return -total_score

# Функция для вычисления вероятности покупки продукта
def purchase_likelihood(price_sensitivity, popularity_sensitivity, analytic_preference, product):
    price_score = -product['PRICE PER UNIT'] * price_sensitivity
    sales_score = (product['SALES VOL'] / max_sales_vol) * popularity_sensitivity
    analytic_score = -abs(product['Analytic1_encoded'] - analytic_preference)

    total_score = price_score + sales_score + analytic_score
    return np.exp(total_score)  # Экспоненциальная функция для нормализации вероятности

# Оптимизация параметров для калибровки модели на исторических данных
def calibration_objective_function(params, data, max_sales_vol):
    price_sensitivity, popularity_sensitivity, analytic_preference = params
    simulated_sales = []

    # Симуляция продаж на основе вероятностей покупки
    for _, product in data.iterrows():
        score = purchase_likelihood(price_sensitivity, popularity_sensitivity, analytic_preference, product)
        simulated_sales.append(score)

    # Приведение продаж к реальному масштабу
    simulated_sales = [s / sum(simulated_sales) * sum(data['SALES VOL']) for s in simulated_sales]

    # Вычисление ошибки модели (MSE)
    error = sum((simulated_sales[i] - data['SALES VOL'].iloc[i]) ** 2 for i in range(len(simulated_sales)))
    return error

# Инициализация DataFrame с продуктами (добавь свой DataFrame 'df')
# df = pd.read_csv('your_data.csv')  # Пример, если данные в csv файле

# Начальные параметры для оптимизации (price_sensitivity, popularity_sensitivity, analytic_preference)
initial_params = [0, 0, 0]

# Определение максимального объема продаж
max_sales_vol = df['SALES VOL'].max()

# Оптимизация параметров модели с вычислением ошибки
result = minimize(calibration_objective_function, initial_params, args=(df, max_sales_vol), method='Nelder-Mead')

# Получение лучших параметров
best_params = result.x
print("Оптимизированные параметры агента:", best_params)

# Вывод финальной ошибки модели
final_error = calibration_objective_function(best_params, df, max_sales_vol)
print("Финальная ошибка модели:", final_error)


Оптимизированные параметры агента: [2.05256687e-04 2.57074914e+00 6.32114496e-01]
Финальная ошибка модели: 54497116756.45829
